In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [ ]:
import pandas as pd

 ## Read the CSV and Perform Basic Data Cleaning    

In [ ]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

## Select your features (columns)     

In [ ]:
# Set features. This will also be used as your x values.
X = df.drop(columns='koi_disposition')
y = df['koi_disposition']

In [ ]:
# Determine which inputs to keep
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(X,y)
#print(model.feature_importances_)

feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(40)

In [ ]:
# After testing, taking the top 16 weighted inputs
# Xsel = df[['koi_fpflag_co', 'koi_fpflag_ss', 'koi_fpflag_nt', 'koi_model_snr', 'koi_fpflag_ec', 
#            'koi_duration_err2', 'koi_steff_err1', 'koi_slogg_err2', 'koi_time0bk_err1', 'koi_teq',
#            'koi_time0bk_err2', 'koi_duration_err1', 'koi_depth', 'koi_depth_err2', 'koi_period',
#            'koi_duration','koi_prad']]
# Xsel
Xsel = X.drop(['koi_impact_err2', 'koi_depth_err1', 'koi_depth_err2', 'koi_insol_err2', 'koi_prad_err2',
              'koi_insol', 'koi_prad', 'koi_prad_err1', 'koi_insol_err1', 'koi_kepmag', 'koi_time0bk',
              'koi_srad', 'koi_steff', 'koi_srad_err2', 'dec', 'ra', 'koi_teq', 'koi_slogg_err1'], axis=1)
Xsel

## Create a Train Test Split     
*Use koi_disposition for the y values*

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xsel, y, random_state = 1, stratify = y)

## Pre-processing     
*Scale the data using the MinMaxScaler and perform some feature selection*

In [ ]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Train the Model     
---------------

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(X_train_scaled, y_train)

In [ ]:
print(f"Training Data Score: {clf.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {clf.score(X_test_scaled, y_test)}")

## Hyperparameter Tuning     
*Use GridSearchCV to tune the model's parameters*

In [ ]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': [5, 7, 8], 'min_weight_fraction_leaf': [.00001],
             'min_samples_split': [0.15, .185]}


grid = GridSearchCV(clf, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
gs = grid.fit(X_train_scaled,y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
predictions = gs.predict(X_test_scaled)

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))